Data path and imports

In [15]:
import pandas as pd
from datetime import datetime
DATA_PATH = "data/room-temperature.csv"

In [19]:
data = pd.read_csv(DATA_PATH)
for (name, r_data) in data.iteritems():
    for val in r_data.values:
        if name == "Date":
            datetime_object = datetime.strptime(val, '%m/%d/%Y %H:%M')
            val = datetime_object.strftime("%p")
            continue
        val -= 273.15
        if val > 23:
            val = 1
        else:
            val = 0
print(data)

                Date  FrontLeft  FrontRight  BackLeft  BackRight
0    4/11/2010 11:30      295.2       297.0     295.8      296.3
1    4/11/2010 12:00      296.2       296.4     296.2      296.3
2    4/11/2010 12:30      297.3       297.5     296.7      297.1
3    4/11/2010 13:00      295.9       296.7     297.4      297.0
4    4/11/2010 13:30      297.2       296.5     297.6      297.4
..               ...        ...         ...       ...        ...
139   4/14/2010 9:00      295.8       294.6     294.8      295.7
140   4/14/2010 9:30      294.8       295.5     294.7      295.6
141  4/14/2010 10:00      295.9       295.8     295.5      295.2
142  4/14/2010 10:30      295.1       296.2     296.0      296.1
143  4/14/2010 11:00      296.2       297.2     296.6      296.0

[144 rows x 5 columns]
